# LangChain 기능 소개

- 1. 프롬프트 템플릿과 로더를 사용하여 체인 구성  
- 2. Runnable과 LangChain 표현 언어  
- 3. RAG 체인 구축  
- 4. 체인이 달린 도구 사용  

## LangChain 개요

LangChain으로 개발된 애플리케이션은 일반적으로 세 가지 범주로 나뉩니다.

- 챗봇 : LLM을 이용하여 보다 지능적인 마케팅, 고객 지원, 교육 상호작용 구현.
- 검색 증강 생성(RAG) Q&A : 대용량 문서 요약, 데이터 분석, 외부 소스를 참조하여 코드 생성에 사용.
- 에이전트 시스템은 다중 에이전트 설정과 인간 상호작용을 포함하며 복잡한 워크플로우를 위해 LangGraph를 활용. 공급망 관리 및 운영 최적화와 같은 분야에 적용 가능.

LangChain은 자연어를 실행 가능한 프로그램으로 변환하는 파이프라인을 생성할 수 있게 합니다. 이러한 체인을 활용함으로써 사용자는 자연어를 입력하고 보고서, 분석 또는 컴퓨터 프로그램과 같은 출력을 받을 수 있습니다.

In [ ]:
# !pip install -U langchain-core langchain
# !pip install -U langchain-openai
# !pip install -U langchain-google-genai
# !pip install -U langchain-community
# !pip install -U langchain-experimental
# !pip install -U langgraph

In [1]:
# env 파일에서 API 키를 로드
from dotenv import load_dotenv
load_dotenv()

True

### 사용 가능한 LLM 목록 조회

## LLM 연결

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage
from openai.error import AuthenticationError

# 모델 초기화
model = init_chat_model("gpt-5-nano", model_provider="openai")

# API Key 유효성 강제 테스트 함수
def validate_key(model):
    try:
        # 아주 짧은 테스트 호출
        response = model.invoke([HumanMessage(content="ping")])
        print("✅ API Key 유효합니다. 테스트 응답:", response.content)
    except AuthenticationError as e:
        print("❌ API Key 인증 실패:", e)
    except Exception as e:
        print("⚠️ 다른 오류 발생:", e)

validate_key(model)


In [2]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-5-nano", model_provider="openai")
# model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [4]:
print(model.invoke("한국어로 LangChain 에 대해서 설명해줘").content)

다음은 한국어로 정리한 LangChain에 대한 설명입니다.

무엇인가요?
- LangChain은 대형언어모델(LLM, 예: GPT-4) 기반의 애플리케이션을 쉽게 만들 수 있도록 돕는 프레임워크예요. 데이터 추출, 문서 요약, 대화형 봇, 코드 보조 등 다양한 LLM 기반 워크플로우를 구성하고 관리하기 위한 도구 모음이라고 보면 됩니다.

주요 구성 요소
- LLM (대형언어모델): 실제로 텍스트를 생성하고 이해하는 모델. OpenAI, Cohere, HuggingFace 등 여러 공급자를 연결해 사용할 수 있어요.
- 프롬프트 템플릿(Prompt Templates): 입력 변수(예: topic, user_id 등)를 활용해 재사용 가능한 프롬프트를 미리 정의하는 방식.
- 체인(Chains): 여러 단계의 작업 흐름을 순차적으로 수행하는 파이프라인. 예를 들어 프롬프트 만들기 -> LLM 호출 -> 결과 파싱 같은 흐름.
- 에이전트(Agents)와 도구(Tools): LLM이 필요에 따라 외부 도구(API)나 데이터에 접근해 실행할 수 있도록 하는 고급 기능. 예: 웹 검색, 계산기, 데이터베이스 질의 등 도구를 사용하도록 지시.
- 메모리(Memory): 대화의 맥락이나 상태를 기억해 오래된 내용도 참고할 수 있게 하는 기능. 대화형 봇의 지속성에 유용.
- 벡터 저장소(Vector Stores)와 임베딩(Embeddings): 문서나 지식을 벡터로 저장하고 필요 시 유사도 검색으로 정보를 회수하는 컴포넌트. Retrieval-Augmented Generation(RAG) 구현에 자주 씁니다.
- 문서 로더/데이터 소스: PDFs, 웹페이지, 파일 시스템 등에서 데이터를 가져와 지식으로 구축하는 도구들.
- 저장 및 재현성: 파이프라인 정의를 코드로 기록하고, 재현 가능하게 실행합니다.

어떤 흐름으로 작동하나요?
- 일반적인 사용 흐름은 다음과 같습니다.
  1) 데이터나 지식을 수집/정리(문서 로더, 임베딩 등).
  2) 프롬프트 템플릿으로 입력 

### Messages
- AIMessage: AI(인공지능) 모델이 생성한 메시지를 나타냅니다.  
예를 들어, AI 모델이 사용자의 질문에 대해 응답을 생성할 때 이 클래스가 사용됩니다.

- HumanMessage: 사람(사용자)이 생성한 메시지를 나타냅니다.  
사용자가 입력한 텍스트나 질문 등이 여기에 해당됩니다.

- StemMessage: AI 행동을 프라이밍하기 위한 메시지.
시스템 메시지는 일반적으로 일련의 입력 메시지 중 첫 번째로 전달됩니다. 일반적으로 대화의 흐름을 제어하거나 특정 지침을 제공하기 위해 사용됩니다.

In [8]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

system_prompt = """
마치 다섯살 먹은 어린아이에게 설명하듯이 한국어로 쉽게 설명해 주세요.
"""

user_prompt = """
LangChain이 무엇인가요?
"""

# system과 human/user 메시지를 이용한 기본 요청

message = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_prompt),
]

message

[SystemMessage(content='\n마치 다섯살 먹은 어린아이에게 설명하듯이 한국어로 쉽게 설명해 주세요.\n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='\nLangChain이 무엇인가요?\n', additional_kwargs={}, response_metadata={})]

In [9]:
response = model.invoke(message)
response

AIMessage(content='LangChain은 똑똑한 앱을 쉽게 만드는 도구 모음이에요. 쉽게 말하면, 큰 언어 모델(예: 챗봇의 뇌)을 이용해서 데이터를 읽고, 도구를 부르고, 기억해 두고, 여러 단계를 순서대로 처리하는 프로그램을 잘 만들 수 있게 도와주는 라이브러리예요.\n\n주요 아이디어를 아주 간단히 비유로 보면:\n- 프롬프트: 로봇에게 무엇을 하라고 지시하는 문구예요.\n- 체인(연쇄): 여러 단계가 차례로 이어지도록 하는 작업 흐름이에요.\n- 에이전트: 상황에 맞게 다음에 무엇을 할지 스스로 결정하는 똑똑한 아이예요.\n- 도구/데이터 소스: 계산기, API, 문서, 데이터베이스 같은 필요한 도구와 자료예요.\n\n쉽게 말해, LangChain은 “로봇이 생각하고, 기억하고, 자료를 찾아보고, 필요하면 API 같은 도구도 쓰면서 일을 차근차근 해내도록” 만들어주는 코드 상자예요.\n\n누가 쓰나요? 개발자들이 파이썬이나 자바스크립트/타입스크립트로 사용해요.\n\n간단한 예: 내부 자료를 찾아 질문에 답하고, 필요하면 캘린더에 일정도 잡아주는 챗봇을 만들고 싶다면, LangChain이 그 흐름을 쉽고 잘 정리해 주는 식이에요.\n\n혹시 어떤 용도나 예제로 궁금한 게 있나요? 구체적으로 알려주시면 그에 맞춰 더 쉽게 설명해 드릴게요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1397, 'prompt_tokens': 42, 'total_tokens': 1439, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 1024, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_token

In [10]:
print(response.content)

LangChain은 똑똑한 앱을 쉽게 만드는 도구 모음이에요. 쉽게 말하면, 큰 언어 모델(예: 챗봇의 뇌)을 이용해서 데이터를 읽고, 도구를 부르고, 기억해 두고, 여러 단계를 순서대로 처리하는 프로그램을 잘 만들 수 있게 도와주는 라이브러리예요.

주요 아이디어를 아주 간단히 비유로 보면:
- 프롬프트: 로봇에게 무엇을 하라고 지시하는 문구예요.
- 체인(연쇄): 여러 단계가 차례로 이어지도록 하는 작업 흐름이에요.
- 에이전트: 상황에 맞게 다음에 무엇을 할지 스스로 결정하는 똑똑한 아이예요.
- 도구/데이터 소스: 계산기, API, 문서, 데이터베이스 같은 필요한 도구와 자료예요.

쉽게 말해, LangChain은 “로봇이 생각하고, 기억하고, 자료를 찾아보고, 필요하면 API 같은 도구도 쓰면서 일을 차근차근 해내도록” 만들어주는 코드 상자예요.

누가 쓰나요? 개발자들이 파이썬이나 자바스크립트/타입스크립트로 사용해요.

간단한 예: 내부 자료를 찾아 질문에 답하고, 필요하면 캘린더에 일정도 잡아주는 챗봇을 만들고 싶다면, LangChain이 그 흐름을 쉽고 잘 정리해 주는 식이에요.

혹시 어떤 용도나 예제로 궁금한 게 있나요? 구체적으로 알려주시면 그에 맞춰 더 쉽게 설명해 드릴게요.


##  1. 프롬프트 템플릿과 로더를 사용하여 체인 구성  

- LLM에서의 Chain 은 Data Processing 에서의 Pipeline 과 유사한 개념

### 체인의 구성 요소 - Runnables
- 프롬프트 : LLM의 응답을 안내하는 템플릿  
- LLM 또는 채팅 모델 : 프롬프트에 따라 응답을 생성하는 엔진  
- 출력 파서 : LLM의 출력을 파싱하는 도구  
- 도구 : LLM이 API에서 추가 정보를 추출하거나 코드를 실행하여 LLM을 에이전트로 전환할 수 있게 해주는 확장 기능  
- 일반 함수 : 서로 연결될 수 있는 추가적인 일반 함수

In [13]:
from langchain.prompts import PromptTemplate

# simple prompt template 생성
# {topic} 변수에서 사용자의 query 가 대체된다.
prompt_template = """
    당신은 AI 주제를 설명하는 데 도움이 되는 어시스턴트입니다. 다음 입력이 주어지면:
    {topic}
    주어진 주제에 대한 설명을 제공하세요.
"""

# prompt template 을 이용하여 prompt 생성 - new style
prompt = PromptTemplate.from_template(prompt_template)
prompt

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='\n    당신은 AI 주제를 설명하는 데 도움이 되는 어시스턴트입니다. 다음 입력이 주어지면:\n    {topic}\n    주어진 주제에 대한 설명을 제공하세요.\n')

In [14]:
# pipe operator "|"" 를 이용하여 하나 이상의 chain 을 결합
chain = prompt | model

print(chain.invoke({"topic": "LangChain이 뭐예요?"}).content)

LangChain은 대형 언어 모델(Large Language Model, LLM)을 활용한 애플리케이션을 쉽게 만들 수 있게 해주는 오픈소스 프레임워크예요. 프롬프트를 여러 단계로 연결하는 “체인(Chain)”과 외부 도구를 사용해 LLM의 능력을 확장하는 “에이전트(Agent)”를 중심으로 구성된 것이 특징입니다.

주요 아이디어
- LLM과 도구를 연결해 단일 프롬프트만으로 해결되지 않는 복잡한 작업을 처리할 수 있습니다.
- 프롬프트 템플릿, 체인, 도구, 메모리(대화 맥락 저장) 등을 한 곳에서 구성하고 재사용할 수 있습니다.
- Python과 TypeScript/JavaScript 환경에서 사용할 수 있습니다.

주요 구성 요소
- LLM: OpenAI, Cohere, HuggingFace 등 다양한 공급자의 모델을 래핑해 사용합니다.
- PromptTemplates: 입력 형식을 미리 정의한 템플릿으로 프롬프트를 구성합니다.
- Chains: LLMChain, SequentialChain 등으로 여러 프롬프트와 로직을 순차적으로 연결합니다.
- Tools: 웹 검색, 계산기, 데이터베이스 질의, 파일 시스템 접근 등 LLM이 스스로 수행하기 어렵던 작업을 수행하는 도구들.
- Agents: 도구를 무엇을 언제 쓸지 결정하는 의사결정 로직으로, ReAct 같은 전략으로 동작하는 경우가 많습니다.
- Memory: 대화 맥락이나 요약 정보를 저장해 다음 상호작용에 활용합니다.
- Vector Stores / Retrieval: 임베딩을 저장하고 필요 시 관련 문서를 검색해 참고합니다.
- LangChain 런타임: Python용 LangChain, TypeScript용 LangChain.js 등 언어별 라이브러리.

작동 원리의 간단한 흐름
- 사용자의 입력이 프롬프트 템플릿으로 변환되고, LLM에 전달됩니다.
- 필요하면 Tools를 호출해 외부 데이터를 얻거나 계산 등을 수행합니다.
- LLM의 출력과 도출된 정보를 다시 프롬프트에 입력해 추가적으

--------------------

## 2. Runnable과 LangChain 표현 언어  

- LCEL(LangChain Expression Language) 는 Runnables 를 chain 으로 구성하는 방법

LCEL은 기본 구성 요소에서 복잡한 체인을 구축하는 것을 간소화합니다. 파이프 연산자(|)를 사용하여 다양한 구성 요소를 체인으로 연결하고 한 요소에서 다음 요소로 출력을 공급합니다. 이런 방식으로 구성된 체인의 간단한 예로는 모델과 출력 파서가 결합된 프롬프트가 있습니다.  이러한 구성 요소들을 runnables 라고 부릅니다.  

`chain=prompt | model | output_parser`

- Chain 구성  
    - 체인에 대한 입력(일반적으로 사전)
    - 입력은 프롬프트로 전송됩니다.
    - 프롬프트 값은 LLM 또는 채팅 모델로 전송됩니다.
    - Chatmodel이 채팅 메시지를 반환합니다.
    - 파서는 채팅 메시지에서 문자열을 추출합니다.
    - 문자열은 체인의 출력입니다

- LangChain의 runnable 객체들:

    - RunnableSequence : 여러 runnable 구성 요소를 연결하여 각 구성 요소가 입력을 처리하고 출력을 다음 구성 요소에 전달
    - RunnableLambda : Python의 호출 가능한 요소(함수 등)를 실행 가능한 구성 요소로 바꿔서 체인으로 통합
    - RunnablePassthrough : 입력을 변경하지 않고 통과시키거나 출력에 추가 키를 추가. placeholder 역할을 하거나 시퀀스에 유연하게 통합할 수 있다.
    - RunnableParallel : 여러 개의 실행 파일을 동시에 실행하여 두 개의 체인이 동일한 입력에서 실행되지만 다른 출력을 반환하는 분기를 허용.

### Runnables

In [16]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt_template = """
    당신은 AI 개념을 요약하는 유용한 조수입니다.
    {context}
    context를 한국어로 요약해 주세요.
"""

summarize_prompt = PromptTemplate.from_template(prompt_template)
summarize_prompt

PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    당신은 AI 개념을 요약하는 유용한 조수입니다.\n    {context}\n    context를 한국어로 요약해 주세요.\n')

###  "|" 연산자를 이용하여 Runnable Sequence chain 생성

In [17]:
output_parser = StrOutputParser()

chain = summarize_prompt | model | output_parser

print(type(chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [18]:
chain.invoke({"context": "LangChain 에 대해 설명해 줘."})

'다음은 LangChain에 대한 간략한 설명과 한국어로의 요약입니다.\n\n- LangChain이란\n  - LLM(대형 언어 모델) 기반 애플리케이션을 쉽게 만들고 운영하기 위한 프레임워크/생태계입니다.\n  - 프롬프트를 체인처럼 연결하는 체인(Chains), 상황에 따라 동적으로 도구를 사용할 수 있게 해주는 에이전트(Agents), 대화나 작업 흐름에서 상태를 기억하는 메모리(Memory), 외부 소스나 도구를 연결하는 도구(Tools) 등 다양한 구성 요소를 제공합니다.\n  - 벡터 저장소, 문서 로더, 임베딩(임베딩 모델), 검색기(Retriever) 같은 기능을 통해 데이터 인입과 검색 기반의 응답을 쉽게 구현할 수 있습니다.\n  - Python용과 JavaScript/TypeScript용 두 가지 구현체가 있어, 서버 사이드나 웹 애플리케이션에 맞춰 사용할 수 있습니다.\n\n- 주요 구성 요소\n  - LLM 래퍼/프로바이더: OpenAI, Cohere, HuggingFace 등 다양한 LLM을 추상화하여 동일한 인터페이스로 조작.\n  - Prompt Templates: PromptTemplate, ChatPromptTemplate 등을 사용해 입력 형식을 표준화하고 재사용 가능한 프롬프트를 만듭니다.\n  - Chains: LLMChain, 데이터 처리를 순차적으로 연결하는 여러 종류의 체인(순차적, 조건부, 순환 등)을 제공합니다.\n  - Agents와 Tools: 에이전트가 상황에 따라 도구(툴)를 호출하고, 필요 시 외부 API나 데이터베이스를 사용할 수 있도록 합니다.\n  - Memory: ConversationalMemory, SummaryMemory 등 대화 맥락이나 장기 기억을 저장하고 활용합니다.\n  - Vector Stores와 Retrievers: FAISS, Pinecone, Weaviate 등 벡터 저장소를 이용해 문서를 임베딩하고 검색합니다.\n  - Document Loaders: PDF, Word, HT

### RunnableLambda 사용

In [19]:
# RunnableLambda를 사용하여 Python 함수를 체인에 삽입합니다.
from langchain_core.runnables import RunnableLambda

summarize_chain = summarize_prompt | model | output_parser
print(type(summarize_chain))

# 사용자 정의 람다 함수를 정의하고 이를 RunnableLambda에 래핑합니다.
length_lambda = RunnableLambda(lambda summary: f"요약된 길이: {len(summary)} 글자")
print(type(length_lambda))

lambda_chain = summarize_chain | length_lambda
print(type(lambda_chain))

<class 'langchain_core.runnables.base.RunnableSequence'>
<class 'langchain_core.runnables.base.RunnableLambda'>
<class 'langchain_core.runnables.base.RunnableSequence'>


In [20]:
lambda_chain.invoke({"context": "LangChain 에 대해 설명해 줘"})

'요약된 길이: 1777 글자'

In [21]:
for step in lambda_chain.steps:
    print(step)

input_variables=['context'] input_types={} partial_variables={} template='\n    당신은 AI 개념을 요약하는 유용한 조수입니다.\n    {context}\n    context를 한국어로 요약해 주세요.\n'
client=<openai.resources.chat.completions.completions.Completions object at 0x10a32bcd0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10a440510> root_client=<openai.OpenAI object at 0x109bac850> root_async_client=<openai.AsyncOpenAI object at 0x109bac3d0> model_name='gpt-5-nano' model_kwargs={} openai_api_key=SecretStr('**********')

RunnableLambda(lambda summary: f'요약된 길이: {len(summary)} 글자')
